In [10]:
import os
import re
import tensorflow as tf

In [11]:
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00
time: 1.3 ms (started: 2024-07-26 23:41:07 +00:00)


In [1]:
!pip install datasets transformers

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
!git lfs install

Git LFS initialized.


In [4]:
!git config --global user.email "aliemre9270@gmail.com"
!git config --global user.name "WhiteAngels"

In [5]:
!git config --global credential.helper store

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
file_name = "test.csv"
path = "/content/gdrive/MyDrive/Colab Notebooks/input/"

In [9]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files= path+file_name)

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 48965
    })
})

In [105]:
dataset['train'][:2]

{'text': ['Kral akbaba dikkat çekici renklere sahiptir .',
  ' ısrarla korkutmayı başarıyor. sanki korku çok uzun ve bitmiyor. zira film bitiyor ama müzik halâ uyarmaya devam ediyor. 9... '],
 'label': ['Notr', 'Positive']}

In [106]:
reviews_sample = dataset["train"].shuffle(seed=42).select(range(45000))

In [107]:
reviews_sample

Dataset({
    features: ['text', 'label'],
    num_rows: 45000
})

In [108]:
reviews_sample = reviews_sample.rename_column(
    original_column_name="text", new_column_name="review"
)
reviews_sample

Dataset({
    features: ['review', 'label'],
    num_rows: 45000
})

In [109]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [110]:
reviews_sample = reviews_sample.map(compute_review_length)
# Inspect the first training example
reviews_sample[0]
reviews_sample.save_to_disk('/content/gdrive/MyDrive/Colab Notebooks/output/reviews_sample_with_lengths')

Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

In [111]:
reviews_sample.sort("review_length")[:3]

{'review': ['#AD?', 'beceri̇kli̇', '2'],
 'label': ['Negative', 'Positive', 'Notr'],
 'review_length': [1, 1, 1]}

In [112]:

import re
# Yorumları temizleme
def remove_repeated(example):
    example["review"] = example["review"].replace('...', '')
    example["review"] = example["review"].replace(',"', '. ')
    example["review"] = example["review"].replace('!.', '.')
    example["review"] = example["review"].replace('!,', '. ')
    example["review"] = example["review"].replace('"', '')
    example["review"] = re.sub('([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ]),([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ])', '\\1. \\2', example["review"])
    return {"review": example["review"].replace('Devamını oku', '')}
reviews_sample = reviews_sample.map(remove_repeated)

# Ara sonuç kaydetme
reviews_sample.save_to_disk('/content/gdrive/MyDrive/Colab Notebooks/output/cleaned_reviews_sample')


Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

In [113]:
reviews_sample = reviews_sample.map(remove_repeated)
reviews_sample[:5]

{'review': ['Ayrıca Gedik Üniversitesi Mütevelli Heyeti üyesidir .',
  ' süper !!! ',
  'James kardeşler gruptan ayrıldılar ve Missouri’ye kaçtılar .',
  ' bence de rezalet bir filmdi. hele ki türkçe dublaj da cabası bu dublaj işi hiç mi hiç olmuyor. tam ilkokul çocuklarına göre bir film. kızın sürekli şarkılar söylemesi insanın sinirine dokunuyor. 10/1 ',
  "daha iyi olabilecek özellikler ise: kablosu pek uzun değil, 50 cm. kadar. ayrıca, kullanma kılavuzunda yanlış bilgi verilmiş. su kaynadıktan sonra kendiliğinden sıcak tutma moduna geçeceği, 0/1 dügmesinin ise 0'a geçeceği yazılmış. aslında 0'a gelmiyor, normal olarak 1'de kalıyor. bu düğme açma kapama düğmesi olduğundan 0'a geldiğinde cihaz aslında kapanıyor. (susuz kaldığında kapanma özelliğini henüz test etmedim)"],
 'label': ['Notr', 'Positive', 'Notr', 'Negative', 'Positive'],
 'review_length': [7, 2, 8, 32, 57]}

In [114]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download tr_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

✘ No compatible package found for 'tr_core_news_sm' (spaCy v3.7.5)



In [24]:
!pip install -U spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_sm
!python -m spacy download tr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebo

In [25]:
import spacy
nlp = spacy.load("en_core_web_sm")


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [26]:
!pip install stanza

In [83]:
import stanza
from transformers import pipeline
import re

# Stanza Türkçe NER modelini indirin ve yükleyin
stanza.download("tr")
nlp_stanza = stanza.Pipeline("tr", processors="tokenize,ner")
# Duygu sınıflandırma pipeline'ını oluşturma
sentiment_pipeline = pipeline("sentiment-analysis", model="emre/turkish-sentiment-analysis", truncation=True, padding=True)

def interpret_sentiment(sentiment_label):
    if sentiment_label == "negative":
        return "negative"
    elif sentiment_label == "neutral":
        return "neutral"
    elif sentiment_label == "positive":
        return "positive"
    else:
        return "neutral"  # Varsayılan olarak duygu durumu yok

# Stop word listesi
stop_words = set([
    "ve", "ile", "de", "da", "bu", "şu", "o", "bir", "daha", "çok", "az", "gibi",
    "için", "mi", "mu", "mü", "mı", "ki", "eğer", "ise", "ama", "fakat", "lakin",
    "ancak", "şayet", "yine", "ise", "hem", "hemde", "bile", "yalnız", "dahi"
])

# Pozitif ve negatif kelime listeleri
positive_words = ["iyi", "güzel", "mükemmel", "harika", "fantastik", "olumlu", "beğendim"]
negative_words = ["kötü", "berbat", "felaket", "rezalet", "olumsuz", "sevmedim"]

def extract_and_classify_entities(review):
    # Metnin uzunluğunu kontrol et ve gerekirse kısalt
    max_length = 512  # Modelin desteklediği maksimum uzunluk

    # Sentiment analizi
    sentiment_result = sentiment_pipeline(review[:max_length])[0]
    sentiment_label = sentiment_result["label"]
    sentiment = interpret_sentiment(sentiment_label)

    # NER analizi
    doc_stanza = nlp_stanza(review[:max_length])
    entities = []

    # Etiketlerin haritalanması
    label_mapping = {
        "PER": "Person_Name",
        "ORG": "Organization_Name",
        "LOC": "Location",
        "GPE": "Location"  # Coğrafi politik birim
    }

    # Stanza'nın tanıdığı varlıklar
    for ent in doc_stanza.ents:
        if ent.type in label_mapping:
            label = label_mapping[ent.type]
            entities.append((ent.text, label, sentiment))

    # Regex ile özel isimleri tanıma (Türkçe isimler için)
    pattern = re.compile(r'\b[A-ZÇĞİÖŞÜ][a-zçğıöşü]*(?:\s+[A-ZÇĞİÖŞÜ][a-zçğıöşü]*)*\b')
    matches = pattern.findall(review)
    for match in matches:
        if match not in [entity[0] for entity in entities]:  # Tekrarları önleme
            entities.append((match, "Person_Name", sentiment))

    # Pozitif ve negatif kelimeleri bağlamlarına göre analiz etme
    words = review.split()
    for i, word in enumerate(words):
        if word.lower() not in stop_words:
            if word.lower() in positive_words or word.lower() in negative_words:
                # Kelimenin bulunduğu cümleyi belirleme
                start = max(0, review[:review.find(word)].rfind('.') + 1)
                end = review.find('.', review.find(word)) + 1
                sentence = review[start:end] if end > start else review

                # Cümlenin duygu durumunu analiz etme
                sentence_sentiment_result = sentiment_pipeline(sentence[:max_length])[0]
                sentence_sentiment_label = sentence_sentiment_result["label"]
                sentence_sentiment = interpret_sentiment(sentence_sentiment_label)

                # Kelimenin duygu durumunu belirleme
                if word.lower() in positive_words and sentence_sentiment == "positive":
                    entities.append((word, "Positive_Word", sentence_sentiment))
                elif word.lower() in negative_words and sentence_sentiment == "negative":
                    entities.append((word, "Negative_Word", sentence_sentiment))

    return {"entities": entities}

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: tr (Turkish) ...
INFO:stanza:File exists: /root/stanza_resources/tr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: tr (Turkish):
| Processor | Package  |
------------------------
| tokenize  | imst     |
| mwt       | imst     |
| ner       | starlang |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [84]:
test_reviews = [
    "Ali ve Ayşe, İstanbul'daki Boğaziçi Üniversitesi'nde eğitim görüyor.",
    "Kocaeli Üniversitesi'nden Dr. Ahmet Yılmaz, yeni bir araştırma yayımladı.",
    "Başkan Recep Tayyip Erdoğan, Cumhurbaşkanlığı Külliyesi'nde bir konuşma yaptı.",
    "Fenerbahçe Spor Kulübü, yeni transferiyle büyük ses getirdi.",
    "Ayla, geçen hafta Bodrum'a tatil yapmaya gitti ve çok keyif aldı.",
    "Ege Üniversitesi'nde yapılan bilimsel çalışmalara katkıda bulunan akademisyenler ödüllendirildi."
]

# Test etme
for review in test_reviews:
    result = extract_and_classify_entities(review)
    print(f"Review: {review}")
    print(f"Entities and Sentiment: {result}\n")


Review: Ali ve Ayşe, İstanbul'daki Boğaziçi Üniversitesi'nde eğitim görüyor.
Entities and Sentiment: {'entities': [('Ali', 'Person_Name', 'neutral'), ('Ayşe', 'Person_Name', 'neutral'), ('İstanbul', 'Person_Name', 'neutral'), ('Boğaziçi Üniversitesi', 'Person_Name', 'neutral')]}

Review: Kocaeli Üniversitesi'nden Dr. Ahmet Yılmaz, yeni bir araştırma yayımladı.
Entities and Sentiment: {'entities': [('Kocaeli Üniversitesi', 'Person_Name', 'neutral'), ('Dr', 'Person_Name', 'neutral'), ('Ahmet Yılmaz', 'Person_Name', 'neutral')]}

Review: Başkan Recep Tayyip Erdoğan, Cumhurbaşkanlığı Külliyesi'nde bir konuşma yaptı.
Entities and Sentiment: {'entities': [('Başkan Recep Tayyip Erdoğan', 'Person_Name', 'neutral'), ('Cumhurbaşkanlığı Külliyesi', 'Person_Name', 'neutral')]}

Review: Fenerbahçe Spor Kulübü, yeni transferiyle büyük ses getirdi.
Entities and Sentiment: {'entities': [('Fenerbahçe Spor Kulübü', 'Person_Name', 'neutral')]}

Review: Ayla, geçen hafta Bodrum'a tatil yapmaya gitti ve ço

In [117]:
# Ara sonuç kaydetme
reviews_sample.save_to_disk('/content/gdrive/MyDrive/Colab Notebooks/output/reviews_with_entities')


Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

In [118]:
reviews_sample[:3]

{'review': ['Ayrıca Gedik Üniversitesi Mütevelli Heyeti üyesidir .',
  ' süper !!! ',
  'James kardeşler gruptan ayrıldılar ve Missouri’ye kaçtılar .'],
 'label': ['Notr', 'Positive', 'Notr'],
 'review_length': [7, 2, 8]}

In [119]:
import stanza
from transformers import pipeline
import re

# Stanza Türkçe NER modelini indirin ve yükleyin
stanza.download("tr")
nlp_stanza = stanza.Pipeline("tr", processors="tokenize,ner")

# Duygu sınıflandırma pipeline'ını oluşturma
sentiment_pipeline = pipeline("sentiment-analysis", model="emre/turkish-sentiment-analysis")


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: tr (Turkish) ...
INFO:stanza:File exists: /root/stanza_resources/tr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: tr (Turkish):
| Processor | Package  |
------------------------
| tokenize  | imst     |
| mwt       | imst     |
| ner       | starlang |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [120]:
# kelime_listesi.py

pozitif_kelimeler = ["güzel", "harika", "mükemmel", "başarılı", "etkileyici", "sürükleyici", "komik", "eğlenceli"]
negatif_kelimeler = ["kötü", "berbat", "sıkıcı", "olumsuz", "üzücü", "sinir bozucu", "şaşırtıcı olmayan"]

stop_words = ["ve", "veya", "bir", "bu", "şu", "o", "ben", "sen", "biz", "siz", "onlar", "için", "ile", "gibi", "ama", "fakat", "çünkü", "rağmen"]

def stop_word_temizle(metin):
  kelimeler = metin.lower().split()
  temizlenmis_kelimeler = [kelime for kelime in kelimeler if kelime not in stop_words]
  return " ".join(temizlenmis_kelimeler)

In [121]:
def interpret_sentiment(sentiment_label):
    if sentiment_label == "negative":
        return "negative"
    elif sentiment_label == "neutral":
        return "neutral"
    elif sentiment_label == "positive":
        return "positive"
    else:
        return "neutral"  # Varsayılan olarak "duygu durumu yok"

In [47]:
!pip install pandas

In [122]:
import pandas as pd
# CSV dosyasını oku
file_name = "test.csv"
path = "/content/gdrive/MyDrive/Colab Notebooks/input/"
full_path = path + file_name

reviews_df = pd.read_csv(full_path)

# Her bir cümleyi analiz et
for review in reviews_df['text'].head(3): # head(3) ile ilk 3 değeri al
    print(f"Review: {review}")
    print(f"Entities and Sentiment: {extract_and_classify_entities(review)}") # Removed review_sample

Review: Kral akbaba dikkat çekici renklere sahiptir .
Entities and Sentiment: {'entities': [('Kral', 'Person_Name', 'neutral')]}
Review:  ısrarla korkutmayı başarıyor. sanki korku çok uzun ve bitmiyor. zira film bitiyor ama müzik halâ uyarmaya devam ediyor. 9... 
Entities and Sentiment: {'entities': []}
Review: Neşe ve Üzüntü köprünün kırılmaya başlamasıyla geri dönerler .
Entities and Sentiment: {'entities': [('Neşe', 'Person_Name', 'neutral'), ('Üzüntü', 'Person_Name', 'neutral')]}


In [125]:
!pip install huggingface_hub
from huggingface_hub import HfApi

reviews_sample.push_to_hub("WhiteAngelss/entity-word-sentiment-analysis")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/45 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/WhiteAngelss/entity-word-sentiment-analysis/commit/d7737ec68389bfd4b7c45a67154f4734727823c7', commit_message='Upload dataset', commit_description='', oid='d7737ec68389bfd4b7c45a67154f4734727823c7', pr_url=None, pr_revision=None, pr_num=None)

In [130]:
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained("redrussianarmy/gpt2-turkish-cased")

# İlk iki yorumu almak
first_two_reviews = reviews_sample[:2]["review"]

outputs = pretrained_tokenizer(
    first_two_reviews,
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {outputs['length']}")
# print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Input IDs length: 2
Input chunk lengths: [7, 3]


In [132]:
print("vocab_size: ", len(pretrained_tokenizer))

vocab_size:  50258


In [133]:
txt = "Sürat Kargom Hala Gelmedi,1402 numaralı kargom adatepe şubesinde."
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)

[11283, 304, 1069, 75, 512, 20172, 2225, 4658, 16, 3168, 5299, 6358, 12546, 512, 989, 3489, 4638, 729, 18]


In [135]:
def get_training_corpus():
    batch_size = 1000
    num_samples = len(reviews_sample)
    return (
        reviews_sample[i : i + batch_size]["review"]
        for i in range(0, num_samples, batch_size)
    )

training_corpus = get_training_corpus()


In [136]:
for reviews in get_training_corpus():
    print(len(reviews))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [137]:
vocab_size = 45000
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

In [138]:
tokenizer.eos_token_id

0

In [139]:
path="./"
file_name="WhiteAngelss/entity-word-sentiment-analysis"
tokenizer.save_pretrained(path+file_name)

('./WhiteAngelss/entity-word-sentiment-analysis/tokenizer_config.json',
 './WhiteAngelss/entity-word-sentiment-analysis/special_tokens_map.json',
 './WhiteAngelss/entity-word-sentiment-analysis/vocab.json',
 './WhiteAngelss/entity-word-sentiment-analysis/merges.txt',
 './WhiteAngelss/entity-word-sentiment-analysis/added_tokens.json',
 './WhiteAngelss/entity-word-sentiment-analysis/tokenizer.json')

In [140]:
loaded_tokenizer = AutoTokenizer.from_pretrained("./WhiteAngelss/entity-word-sentiment-analysis")

In [143]:
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [144]:
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)  # Builds the model
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  120402432 
 er)                                                             
                                                                 
Total params: 120402432 (459.30 MB)
Trainable params: 120402432 (459.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


TypeError: The dataset length is unknown.

In [128]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(vocab_size=30000, min_frequency=2)
tokenizer.train_from_iterator(training_corpus, trainer)

tokenizer.save("/content/gdrive/MyDrive/Colab Notebooks/input/tokenizer.json")


In [195]:
import tensorflow as tf
from transformers import create_optimizer, AutoTokenizer, TFGPT2LMHeadModel, AutoConfig
from datasets import load_dataset

# Google Drive bağlantısı
from google.colab import drive
drive.mount('/content/gdrive')

# CSV dosyasını yükleme
file_name = "test.csv"
path = "/content/gdrive/MyDrive/Colab Notebooks/input/"
dataset = load_dataset("csv", data_files=path + file_name)

# Define the tokenizer here
loaded_tokenizer = AutoTokenizer.from_pretrained("./WhiteAngelss/entity-word-sentiment-analysis")

# Eğitim veri kümesini TensorFlow veri kümesine dönüştürme
def dataset_to_tf(dataset, batch_size=32):
    def gen():
        for ex in dataset:
            # Tokenize the review text
            tokens = loaded_tokenizer(ex["review"], return_tensors="tf", padding="max_length", truncation=True)
            yield tokens.input_ids, ex["label"]

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(None, 128), dtype=tf.int32),  # Adjust shape if necessary
            tf.TensorSpec(shape=(), dtype=tf.int64),
        ),
    ).batch(batch_size)

# TensorFlow veri kümesini oluşturma
tf_train_dataset = dataset_to_tf(dataset["train"])

# Calculate the length of the dataset before batching
num_train_steps = len(list(dataset["train"]))

# Optimizasyon ayarları
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

# Define the model
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(loaded_tokenizer),
    n_ctx=128, # You might need to adjust this based on your data
    bos_token_id=loaded_tokenizer.bos_token_id,
    eos_token_id=loaded_tokenizer.eos_token_id,
)
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)  # Builds the model



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TFCausalLMOutputWithCrossAttentions(loss=None, logits=<tf.Tensor: shape=(1, 2, 45000), dtype=float16, numpy=
array([[[-0.0667, -0.1316, -0.2222, ..., -0.745 , -0.2678, -0.3625],
        [ 0.1621, -0.668 ,  0.1227, ..., -0.5625, -0.4282, -0.321 ]]],
      dtype=float16)>, past_key_values=(<tf.Tensor: shape=(2, 1, 12, 2, 64), dtype=float16, numpy=
array([[[[[ 0.5054  ,  0.9316  , -0.1207  , ..., -0.3306  ,  0.6616  ,
            0.1721  ],
          [-0.2213  ,  0.8584  , -0.10986 , ..., -0.01282 ,  0.1067  ,
           -0.1102  ]],

         [[ 0.7314  ,  0.3608  ,  0.3154  , ..., -0.3918  ,  0.3306  ,
            0.07715 ],
          [ 0.284   ,  0.05725 ,  0.879   , ..., -0.1912  , -0.2322  ,
            0.4194  ]],

         [[ 0.1626  ,  0.3086  , -0.08923 , ..., -0.3794  ,  0.3208  ,
            0.3428  ],
          [ 0.4785  ,  0.4102  , -0.509   , ...,  0.3389  , -0.1841  ,
            0.421   ]],

         ...,

         [[ 0.203   , -0.0659  ,  0.801   , ..., -0.1033  ,  0.5337

In [196]:
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [191]:
def dataset_to_tf(dataset, batch_size=32):
    def gen():
        for ex in dataset:
            # Tokenize the review text
            tokens = loaded_tokenizer(ex["review"], return_tensors="tf", padding="max_length", truncation=True, max_length=128)  # Ensure correct max_length
            # Ensure the label is a one-dimensional tensor with the correct shape
            label = tf.constant([ex["label"]], dtype=tf.int64)  # Create a constant tensor with the correct shape
            yield tokens.input_ids, label

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(None, 128), dtype=tf.int32),  # Adjust shape according to your model's requirements
            tf.TensorSpec(shape=(None, 1), dtype=tf.int64),  # The label should have shape (None, 1)
        ),
    ).batch(batch_size)

In [183]:
# TensorFlow veri kümesini oluşturma
tf_train_dataset = dataset_to_tf(dataset["train"])

# Check if 'test' split exists
if 'test' in dataset:
  tf_eval_dataset = dataset_to_tf(dataset["test"])

  # Eğitim sürecini başlatma
  history = model.fit(
      tf_train_dataset,
      validation_data=tf_eval_dataset,
      epochs=3,
      callbacks=[]  # İhtiyacınıza göre callbacks ekleyin
  )
else:
  print("No evaluation split found in the dataset.")

No evaluation split found in the dataset.
